# Local RAG agent with LLaMA3

#### Installation  

* pip install ollama langchain beautifulsoup4 chromadb gradio
* ollama pull llama3
* ollama pull nomic-embed-text

In [17]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [11]:
# Loading the data using a WebBaseLoader 

# Passing the URL as the input

loader = WebBaseLoader("https://www.investopedia.com/terms/e/environmental-social-and-governance-esg-criteria.asp")

docs = loader.load()

# Splitting the loaded text into chunks of size 1000 with an overlap of 200
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)


In [3]:
# Creating an Ollama embeddings and vector store to store the chunks

embeddings = OllamaEmbeddings(model="nomic-embed-text") # using nombic-embed-text model for embedding

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


In [4]:
# Defining Ollama Llama3 model

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question} \n\n Context: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

In [5]:
# RAG Setup

retriever = vectorstore.as_retriever()

def combine_docs(docs):
    """ 
    Combines retrieved documents which are in mulitple splits or chunks into a single document.
    """
    return "\n\n".join(doc.page_content for doc in docs)

def rag_chain(question):
    """
    Calls ollama llama3 model and the retrieved documents and question are passed as arguments to it.
    """
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)


In [17]:
# RAG App

result = rag_chain("What is ESG investing?")
print(result)

ESG stands for Environmental, Social, and Governance. It refers to a type of investing that focuses on companies that follow positive principles in these areas. ESG investing evaluates a company's performance based on its environmental impact, social responsibility, and governance practices.

Environmental criteria assess how well a company protects the environment, such as by reducing carbon emissions or conserving natural resources.

Social criteria examine how a company treats its employees, suppliers, customers, and communities, including issues like diversity and inclusion, labor standards, and human rights.

Governance measures a company's leadership, executive pay, audits, internal controls, and shareholder rights.


### Performing the same by calling Llama3 model from ChatOllama

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama # to use llama3 model 


The Prompt Template for Llama 3 is bit different attaching the documentation link to it

'https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/'

In [22]:

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model='llama3', temperature=0)

# Post-processing 
def format_docs(docs):
    """ 
    Formats retrieved documents which are in mulitple splits or chunks into a single document.
    """
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "What is ESG Investing?"

docs = retriever.invoke(question)

generation = rag_chain.invoke({"context": docs, "question": question})

print(generation)

ESG investing refers to a set of standards that socially conscious investors use to screen investments. It focuses on companies that follow positive environmental, social, and governance principles.


# Built an UI using Gradio

In [14]:
from Gradio_UI import launch_gradio

In [16]:
launch_gradio()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
